<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#安裝套件" data-toc-modified-id="安裝套件-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>安裝套件</a></span></li><li><span><a href="#define-function" data-toc-modified-id="define-function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>define function</a></span><ul class="toc-item"><li><span><a href="#何謂深度擺盤？" data-toc-modified-id="何謂深度擺盤？-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>何謂深度擺盤？</a></span></li></ul></li><li><span><a href="#Demo" data-toc-modified-id="Demo-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Demo</a></span></li></ul></div>

# 安裝套件

In [4]:
#安裝相關套件
import math
import pandas as pd
import requests
import threading
import schedule
import sys
import datetime

# 安裝視覺化套件
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


# define function

In [5]:
# 透過 meta API 取得個股基本資訊

def transfer_symbolName(symbolId,apiToken):
    
    meta_query = requests.get(f'https://api.fugle.tw/realtime/v0.3/intraday/meta?symbolId={str(symbolId)}&apiToken={apiToken}')
 
    symbol_name = meta_query.json()['data']['meta']['nameZhTw']
    
    return symbol_name



## 何謂深度擺盤？

「深度擺盤」簡單來說就是當您想要推升一檔股票到某個價位時，直接幫您計算出可能需要準備多少資金，才能進行推價操作。

In [ ]:
# 深度擺盤計算

def transfer_cumVolume(asks_data, bids_data):
    
    new_asks_list = []
    new_bids_list = []

    asks_volume_data = bids_volume_data = asks_amount_data = bids_amount_data = 0

    for i in range(len(asks_data)):

        asks_price_data = asks_data[i]['price']
        asks_volume_data += asks_data[i]['volume']
        asks_amount_data += asks_data[i]['price']*asks_data[i]['volume']/10 #萬元

        bids_price_data = bids_data[i]['price']
        bids_volume_data += bids_data[i]['volume']
        bids_amount_data += bids_data[i]['price']*bids_data[i]['volume']/10 #萬元


        new_asks_list.append({'price':asks_price_data, 'volume':asks_volume_data ,'amount': asks_amount_data})

        new_bids_list.append({'price':bids_price_data, 'volume':bids_volume_data, 'amount': bids_amount_data})
    
    return new_asks_list, new_bids_list


# Demo

In [8]:
apiToken = 'demo' # 輸入您的 Token

In [9]:
## 比較 n 檔個股的深度擺盤_v2

app = dash.Dash(__name__)

app.layout = html.Div(['輸入股票代碼：',
    dcc.Input(
        id = "input_symbol1",
        type="text",
        value = 2884),                    
    
    html.Div([
        dcc.Graph(id = 'output_figure'),   
        ]),
    
    html.Div([
        dcc.Graph(id = 'output_figure2'),   
        ]),
    
    html.Div(id='live-update-text'),
    
    dcc.Interval(id = 'fig-update',
             interval = 3*1000, # 5*60s update
             n_intervals = 0)
    
])

now_time = datetime.datetime.now()

@app.callback(Output('live-update-text', 'children'),
              [dash.dependencies.Input('fig-update', 'n_intervals')])
def update_metrics(test):
    
    style = {'padding': '5px', 'fontSize': '16px'}
    updateTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return [
        html.Span(f'最新更新時間：{updateTime}', style = style),
    ]


@app.callback(
    [Output("output_figure", "figure"),
     Output("output_figure2", "figure"),],
    [dash.dependencies.Input('fig-update', 'n_intervals'),
     Input("input_symbol1", "value"),]

)


def get_figure(n, symbol1):
 
    print('更新資料中...')

    # 繪製即時走勢+成交明細
    
    # 透過 chart API 繪製即時走勢圖
    price_col = requests.get(f"https://api.fugle.tw/realtime/v0.3/intraday/chart?symbolId={str(symbol1)}&apiToken={apiToken}")
    
    
    # 透過 dealts API 取得成交明細預計呈現的欄位
    dealts_col = requests.get(f"https://api.fugle.tw/realtime/v0.3/intraday/dealts?symbolId={str(symbol1)}&apiToken={apiToken}&limit=500")
    dealts_data = pd.DataFrame(dealts_col.json()['data']['dealts'])
    dealts_data['at'] = list(map(lambda x : x[11:19],dealts_data['at']))
    
    # 取得預計使用的欄位
    dealts_data = dealts_data[['at', 'bid', 'ask', 'price', 'volume']].rename(columns={'at':'時間','bid':'買進','ask':'賣出','price':'成交','volume':'量'})   


    fig = make_subplots(rows=1, cols=2,    
                        specs=[[{"type":"scatter"},{"type": "table"}],
                        ],
                        subplot_titles=('即時走勢圖','成交明細表',))

    fig.add_trace(go.Scatter(x= list(map(lambda x: datetime.datetime.fromtimestamp(x/1000), price_col.json()['data']['chart']['t'])),
                             y=price_col.json()['data']['chart']['c'],
                             name='即時走勢',
                             marker_color = '#4865E8',
                             mode="lines",
                             textposition="top center"),
                             row = 1, col = 1) # fill down to xaxis

    fig.add_trace(
        go.Table(
            header=dict(
                values=dealts_data.columns.tolist(),
                font=dict(size=10),
                align="left"
            ),
            cells=dict(
                values=[dealts_data[k].tolist() for k in dealts_data.columns.tolist()],
                align = "left")
        ),
        row=1, col=2
    )    
    
    fig.update_xaxes(range = [datetime.datetime(now_time.year, now_time.month, now_time.day, 9,0,0),
                              datetime.datetime(now_time.year, now_time.month, now_time.day, 13,30,0)],
        row=1, col=1)
    
    fig.update_layout(
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  height = 500,
                  font = dict(size = 13))    
    
    
    #繪製深度擺盤圖
    fig2 = go.Figure()
    
    # 透過 quote API 取得最新五檔資訊進行深度擺盤繪製
    quote_url = f"https://api.fugle.tw/realtime/v0.3/intraday/quote?symbolId={str(symbol1)}&apiToken={apiToken}"

    quote_data = requests.get(quote_url)

    # 委賣
    asks_data = quote_data.json()['data']['quote']['order']['asks']

    # 委買
    bids_data = quote_data.json()['data']['quote']['order']['bids']
    
    new_asks_list, new_bids_list = transfer_cumVolume(asks_data, bids_data)
    
    bids_price = list(map(lambda x:x['price'],new_bids_list))
    asks_price = list(map(lambda x:x['price'],new_asks_list))

    price_range = (max(bids_price)-min(bids_price))/4


    fig2.add_trace(go.Scatter(x=bids_price,
                              y=list(map(lambda x:x['volume'], new_bids_list)), fill='tozeroy',name='委買',
                              marker_color = '#4865E8',
                              text= list(map(lambda x:str(round(x['amount'],2))+'萬元', new_bids_list)),
                              mode="lines+markers+text",
                              textposition="top center")) # fill down to xaxis
    
    fig2.add_trace(go.Scatter(x=asks_price,
                              y=list(map(lambda x:x['volume'], new_asks_list)), fill='tozeroy',name='委賣',
                              marker_color = '#FFA84A',
                              text= list(map(lambda x:str(round(x['amount'],2))+'萬元', new_asks_list)),
                              mode="lines+markers+text",
                              textposition="top center")) # fill to trace0 y


  
    fig2.update_xaxes(range = [min(bids_price)-price_range, max(asks_price)+price_range])

    
    fig2.update_layout(title = f'{transfer_symbolName(symbol1, apiToken)} ({str(symbol1)}.tw) 深度擺盤圖',
                  uniformtext_minsize=8, uniformtext_mode='hide',
                  height = 500,
                  font = dict(size = 13))
    
    print('更新完成')
    
    return [fig, fig2]


if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=8053)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8053/ (Press CTRL+C to quit)
220.135.2.3 - - [17/Mar/2022 14:34:29] "POST /_dash-update-component HTTP/1.1" 200 -


更新資料中...
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/alanlin/.local/lib/python3.8/s

220.135.2.3 - - [17/Mar/2022 14:34:29] "POST /_dash-update-component HTTP/1.1" 500 -
220.135.2.3 - - [17/Mar/2022 14:34:32] "POST /_dash-update-component HTTP/1.1" 200 -


更新資料中...
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/alanlin/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/alanlin/.local/lib/python3.8/s

...

更新資料中...
更新完成


220.135.2.3 - - [17/Mar/2022 14:38:31] "POST /_dash-update-component HTTP/1.1" 200 -
220.135.2.3 - - [17/Mar/2022 14:38:31] "POST /_dash-update-component HTTP/1.1" 200 -


更新資料中...
更新完成


220.135.2.3 - - [17/Mar/2022 14:38:34] "POST /_dash-update-component HTTP/1.1" 200 -
220.135.2.3 - - [17/Mar/2022 14:38:34] "POST /_dash-update-component HTTP/1.1" 200 -


更新資料中...
更新完成
